In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import numpy as np
import os
import json
import math
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import gc

# === 激活 DirectML (Windows AMD) ===
try:
    import torch_directml
    DEVICE = torch_directml.device()
    print(f"✅ 成功激活 DirectML 加速设备: {DEVICE}")
except ImportError:
    DEVICE = torch.device("cpu")
    print("❌ 未检测到 torch-directml，使用 CPU")

# ================= 配置区域 =================
DATA_PATH = "../output/chess_transformer/dataset_seq_strict.csv" # 确保指向最新的清洗版数据
CHECKPOINT_DIR = "../output/chess_transformer/checkpoints_dual"  # 改个目录名，区分单输出版本
MODEL_NAME = "chess_transformer_dual.pth"
VOCAB_FILE = "vocab.json"

CHUNK_SIZE = 200000    
BATCH_SIZE = 512       
D_MODEL = 256
N_HEAD = 8
NUM_LAYERS = 4
LEARNING_RATE = 1e-4
MAX_SEQ_LEN = 40
EPOCHS = 10


✅ 成功激活 DirectML 加速设备: privateuseone:0


In [9]:

# ================= 1. 词表与数据处理 =================

class ChessVocab:
    def __init__(self):
        self.move2id = {"<PAD>": 0, "<UNK>": 1}
        self.id2move = {0: "<PAD>", 1: "<UNK>"}
        
    def build_from_file(self, csv_path):
        print("🚀 正在扫描大文件构建词表...")
        unique_moves = set()
        chunk_iter = pd.read_csv(csv_path, chunksize=CHUNK_SIZE)
        for i, chunk in enumerate(chunk_iter):
            print(f"  扫描 Chunk {i+1}...", end='\r')
            chunk = chunk.dropna(subset=['Moves'])
            moves_list = chunk['Moves'].str.split()
            for moves in moves_list:
                unique_moves.update(moves)
        
        print(f"\n✅ 扫描完成！发现 {len(unique_moves)} 种唯一棋步")
        for move in sorted(list(unique_moves)):
            if move not in self.move2id:
                idx = len(self.move2id)
                self.move2id[move] = idx
                self.id2move[idx] = move
        
    def encode(self, moves_list):
        return [self.move2id.get(m, 1) for m in moves_list]

    def save(self, path):
        with open(path, 'w') as f:
            json.dump(self.move2id, f)

    def load(self, path):
        with open(path, 'r') as f:
            self.move2id = json.load(f)
        self.id2move = {v: k for k, v in self.move2id.items()}

# === 修改点 1: Dataset 读取双目标 ===
class ChessDataset(Dataset):
    def __init__(self, df, vocab, max_len=40):
        self.vocab = vocab
        self.max_len = max_len
        self.data = []
        self.targets = []
        
        # 同时读取 White 和 Black 的 ELO
        w_elos = df['WhiteElo'].values.astype(np.float32)
        b_elos = df['BlackElo'].values.astype(np.float32)
        moves_series = df['Moves'].str.split()
        
        for moves, w, b in zip(moves_series, w_elos, b_elos):
            moves = moves[:max_len]
            token_ids = self.vocab.encode(moves)
            self.data.append(torch.tensor(token_ids, dtype=torch.long))
            # Target 变成了 [white_elo, black_elo] 的列表
            self.targets.append([w, b])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

# === 修改点 2: Collate 函数适配双目标形状 ===
def collate_fn(batch):
    inputs, targets = zip(*batch)
    inputs_padded = pad_sequence(inputs, batch_first=True, padding_value=0)
    
    # targets 现在是 list of lists [[w1, b1], [w2, b2], ...]
    # 转换为 tensor shape [batch_size, 2]
    targets_tensor = torch.tensor(targets, dtype=torch.float32)
    
    padding_mask = (inputs_padded == 0)
    return inputs_padded, targets_tensor, padding_mask


In [10]:

# ================= 2. 模型定义 =================

class ChessTransformer(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_layers, max_len=40):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=0)
        self.pos_embedding = nn.Parameter(torch.randn(1, max_len, d_model))
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=d_model*4, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # === 修改点 3: 输出层改为 2 ===
        self.fc = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Linear(d_model // 2, 2) # Output: [White_ELO, Black_ELO]
        )

    def forward(self, x, src_key_padding_mask):
        seq_len = x.size(1)
        x = self.embedding(x) + self.pos_embedding[:, :seq_len, :]
        x = self.transformer(x, src_key_padding_mask=src_key_padding_mask)
        x = x.mean(dim=1) 
        out = self.fc(x)
        return out


In [11]:

# ================= 3. 工具函数 =================

def save_checkpoint(model, optimizer, epoch, chunk_idx, loss, vocab, dir_path):
    if not os.path.exists(dir_path): os.makedirs(dir_path)
    ckpt_path = os.path.join(dir_path, MODEL_NAME)
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }, ckpt_path)
    vocab.save(os.path.join(dir_path, VOCAB_FILE))

def load_checkpoint(model, optimizer, dir_path):
    ckpt_path = os.path.join(dir_path, MODEL_NAME)
    vocab_path = os.path.join(dir_path, VOCAB_FILE)
    start_epoch = 0
    vocab = ChessVocab()
    
    if os.path.exists(ckpt_path) and os.path.exists(vocab_path):
        print(f"🔄 加载旧模型: {ckpt_path}")
        checkpoint = torch.load(ckpt_path, map_location=DEVICE)
        
        # 简单的权重形状检查，防止加载单输出模型报错
        try:
            model.load_state_dict(checkpoint['model_state_dict'])
        except RuntimeError as e:
            print("⚠️ 警告: 存档与当前模型结构不匹配 (可能是单输出旧档?)")
            print("正在重新开始训练...")
            return 0, vocab

        if optimizer: optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch']
        vocab.load(vocab_path)
    else:
        print("✨ 开始新训练")
    return start_epoch, vocab

# Python 版严格清洗 (必须与 C++ 一致)
def clean_strict(move_str):
    raw = move_str
    if raw.startswith("O-O"):
        if raw.startswith("O-O-O"): return "O-O-O"
        return "O-O"
    rank_idx = -1
    for i in range(len(raw) - 1, -1, -1):
        if raw[i].isdigit():
            rank_idx = i
            break
    if rank_idx <= 0: return ""
    dest = raw[rank_idx-1 : rank_idx+1]
    first = raw[0]
    piece = ""
    if first.isupper(): piece = first
    promotion = ""
    if "=" in raw:
        idx = raw.find("=")
        if idx + 1 < len(raw): promotion = "=" + raw[idx+1]
    return piece + dest + promotion


In [12]:

# ================= 4. 训练逻辑 =================

def train_dual():
    if not os.path.exists(DATA_PATH):
        print("❌ 找不到数据文件")
        return

    vocab = ChessVocab()
    vocab_path = os.path.join(CHECKPOINT_DIR, VOCAB_FILE)
    if not os.path.exists(CHECKPOINT_DIR): os.makedirs(CHECKPOINT_DIR)

    if os.path.exists(vocab_path):
        vocab.load(vocab_path)
    else:
        vocab.build_from_file(DATA_PATH)
        vocab.save(vocab_path)

    model = ChessTransformer(len(vocab.move2id), D_MODEL, N_HEAD, NUM_LAYERS).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.MSELoss() # MSE 同样适用于多输出

    start_epoch, _ = load_checkpoint(model, optimizer, CHECKPOINT_DIR)

    for epoch in range(start_epoch, EPOCHS):
        print(f"\n=== Epoch {epoch + 1}/{EPOCHS} ===")
        chunk_iterator = pd.read_csv(DATA_PATH, chunksize=CHUNK_SIZE)
        
        total_epoch_loss = 0
        chunks_count = 0
        
        for i, chunk in enumerate(chunk_iterator):
            chunk = chunk.dropna(subset=['Moves'])
            if len(chunk) == 0: continue
            
            train_df, val_df = train_test_split(chunk, test_size=0.05)
            
            train_dataset = ChessDataset(train_df, vocab)
            train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn, num_workers=0)
            
            model.train()
            chunk_loss = 0
            pbar = tqdm(train_loader, desc=f"Ep {epoch+1} | Chunk {i+1}", leave=False)
            
            for inputs, targets, mask in pbar:
                inputs, targets, mask = inputs.to(DEVICE), targets.to(DEVICE), mask.to(DEVICE)
                
                optimizer.zero_grad()
                outputs = model(inputs, mask) # Output shape: [batch, 2]
                loss = criterion(outputs, targets) # Target shape: [batch, 2]
                loss.backward()
                optimizer.step()
                
                chunk_loss += loss.item()
                pbar.set_postfix({'rmse': math.sqrt(loss.item())})
            
            avg_chunk_loss = chunk_loss / len(train_loader)
            total_epoch_loss += avg_chunk_loss
            chunks_count += 1
            
            del train_dataset, train_loader, train_df, val_df, chunk
            gc.collect() 
            
            if (i + 1) % 10 == 0:
                save_checkpoint(model, optimizer, epoch, i, avg_chunk_loss, vocab, CHECKPOINT_DIR)
                print(f"  [保存] Chunk {i+1} 完成")

        avg_epoch_loss = total_epoch_loss / chunks_count if chunks_count > 0 else 0
        print(f"Epoch {epoch+1} 完成. 平均 RMSE: {math.sqrt(avg_epoch_loss):.2f}")
        save_checkpoint(model, optimizer, epoch + 1, 0, avg_epoch_loss, vocab, CHECKPOINT_DIR)


In [13]:

# ================= 5. 预测接口 =================

def predict_dual(raw_pgn):
    vocab = ChessVocab()
    vocab_path = os.path.join(CHECKPOINT_DIR, VOCAB_FILE)
    if not os.path.exists(vocab_path):
        print("请先训练！")
        return

    vocab.load(vocab_path)
    model = ChessTransformer(len(vocab.move2id), D_MODEL, N_HEAD, NUM_LAYERS).to(DEVICE)
    load_checkpoint(model, None, CHECKPOINT_DIR)
    model.eval()
    
    # 清洗 + 编码
    raw_moves = raw_pgn.split()
    clean_moves = [clean_strict(m) for m in raw_moves]
    # 过滤掉空串 (比如只有评论的情况)
    clean_moves = [m for m in clean_moves if m]
    clean_moves = clean_moves[:MAX_SEQ_LEN]
    
    token_ids = vocab.encode(clean_moves)
    tensor_in = torch.tensor(token_ids, dtype=torch.long).unsqueeze(0).to(DEVICE)
    mask = torch.zeros((1, len(token_ids)), dtype=torch.bool).to(DEVICE)
    
    with torch.no_grad():
        preds = model(tensor_in, mask) # [1, 2]
        w_pred = preds[0][0].item()
        b_pred = preds[0][1].item()
        
        print(f"\n输入对局: {' '.join(clean_moves[:10])}...")
        print(f"预测 White ELO: {w_pred:.0f}")
        print(f"预测 Black ELO: {b_pred:.0f}")
        print(f"分差预测: {abs(w_pred - b_pred):.0f}")


In [ ]:
train_dual() # 训练模式


🚀 正在扫描大文件构建词表...


In [ ]:

# 预测模式示例
# test_game = "e4 c5 Nf3 d6 d4 cxd4 Nxd4 Nf6 Nc3 a6 Bg5 e6 f4 Qb6" # 复杂的西西里防御
# predict_dual(test_game)